In [2]:
import torch
import scvi
import pandas as pd
import scanpy as sc
from scvi import REGISTRY_KEYS
import scgen
from captum.attr import FeatureAblation

sc.set_figure_params(dpi=100, frameon=False, color_map='Reds', facecolor=None)
sc.logging.print_header()

/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/torch/utils/tensorboard/__init__.py:2: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.version import LooseVersion
Global seed set to 0
/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/captum/attr/_utils/visualization.py:15: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import HTML, display
/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/scanpy/_settings.py:447: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  IPython.display.set_matplotlib_formats(*ipython_format)


scanpy==1.9.1 anndata==0.8.0 umap==0.5.3 numpy==1.22.4 scipy==1.9.1 pandas==1.4.4 scikit-learn==1.1.2 statsmodels==0.13.2 pynndescent==0.5.7


/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/umap/__init__.py:9: ImportWarning: Tensorflow not installed; ParametricUMAP will be unavailable
  warn(


## load model and data

In [3]:
base_path = '/home/icb/yuge.ji/projects/feature-attribution-sc'
adata = sc.read(f'{base_path}/datasets/scgen_norman19.h5ad')
adata

/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/anndata/_io/h5ad.py:238: OldFormatWarning: Element '/layers' was written without encoding metadata.
  d[k] = read_elem(f[k])
/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/anndata/_io/specs/methods.py:584: OldFormatWarning: Element '/obs/__categories/guide_identity' was written without encoding metadata.
  categories = read_elem(categories_dset)
/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/anndata/_io/specs/methods.py:587: OldFormatWarning: Element '/obs/guide_identity' was written without encoding metadata.
  read_elem(dataset), categories, ordered=ordered
/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/anndata/_io/specs/methods.py:590: OldFormatWarning: Element '/obs/read_count' was written without encoding metadata.
  return read_elem(dataset)
/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/anndata/_io/specs/

/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/anndata/_io/h5ad.py:238: OldFormatWarning: Element '/obsm' was written without encoding metadata.
  d[k] = read_elem(f[k])
/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/anndata/_io/specs/methods.py:92: OldFormatWarning: Element '/obsm/X_pca' was written without encoding metadata.
  return {k: read_elem(v) for k, v in elem.items()}
/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/anndata/_io/specs/methods.py:92: OldFormatWarning: Element '/obsm/X_umap' was written without encoding metadata.
  return {k: read_elem(v) for k, v in elem.items()}
/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/anndata/_io/h5ad.py:238: OldFormatWarning: Element '/obsp' was written without encoding metadata.
  d[k] = read_elem(f[k])
/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/anndata/_io/h5ad.py:238: OldFormatWarning: Element '/uns' wa

AnnData object with n_obs × n_vars = 69570 × 19018
    obs: 'guide_identity', 'read_count', 'UMI_count', 'coverage', 'gemgroup', 'good_coverage', 'number_of_cells', 'guide_AHR', 'guide_ARID1A', 'guide_ARRDC3', 'guide_ATL1', 'guide_BAK1', 'guide_BCL2L11', 'guide_BCORL1', 'guide_BPGM', 'guide_C19orf26', 'guide_C3orf72', 'guide_CBFA2T3', 'guide_CBL', 'guide_CDKN1A', 'guide_CDKN1B', 'guide_CDKN1C', 'guide_CEBPA', 'guide_CEBPB', 'guide_CEBPE', 'guide_CELF2', 'guide_CITED1', 'guide_CKS1B', 'guide_CLDN6', 'guide_CNN1', 'guide_CNNM4', 'guide_COL1A1', 'guide_COL2A1', 'guide_CSRNP1', 'guide_DLX2', 'guide_DUSP9', 'guide_EGR1', 'guide_ELMSAN1', 'guide_ETS2', 'guide_FEV', 'guide_FOSB', 'guide_FOXA1', 'guide_FOXA3', 'guide_FOXF1', 'guide_FOXL2', 'guide_FOXO4', 'guide_GLB1L2', 'guide_HES7', 'guide_HK2', 'guide_HNF4A', 'guide_HOXA13', 'guide_HOXB9', 'guide_HOXC13', 'guide_IER5L', 'guide_IGDCC3', 'guide_IKZF3', 'guide_IRF1', 'guide_ISL2', 'guide_JUN', 'guide_KIAA1804', 'guide_KIF18B', 'guide_KIF2C', 'g

In [4]:
file = 'scgen_norman19_model1_shuffled'
model = scgen.SCGEN.load(f'{base_path}/models/{file}', adata=adata)
model

INFO     File /home/icb/yuge.ji/projects/feature-attribution-sc/models/scgen_norman19_model1_
         shuffled/model.pt already downloaded                                                


SCGEN Model with the following params: 
n_hidden: 800, n_latent: 100, n_layers: 2, dropout_rate: 0.2
Training status: Trained

get cell type names that match the labels (integers in the model):

In [ ]:
adata.

In [5]:
# ct_names = model.adata_manager.get_state_registry(REGISTRY_KEYS.LABELS_KEY)['original_key']
# ct_names = [ct for ct in ct_names if ct != "unlabeled"]

retrieve the data as a tensor from the dataloader. We'll define batch size to be the size of the whole dataset to do so.

We also need to subset Norman to achieve a reasonable runtime given a model trained on the full gene set, as this feature ablation implementation scales with the number of features.

In [47]:
adata.obs.perturbation_name.value_counts()

control    11835
KLF1        1954
BAK1        1451
CEBPE       1230
ETS2        1201
           ...  
FOXO4        215
NIT1         191
ZBTB10       160
HES7         126
PLK4         113
Name: perturbation_name, Length: 106, dtype: int64

In [58]:
import numpy 
indices = [np.array(sc.pp.subsample(adata[adata.obs.perturbation_name == c], n_obs=100, copy=True).obs.index) for c in adata.obs.perturbation_name.unique()]

NameError: name 'np' is not defined

In [ ]:
indice

In [51]:
import anndata as ad
indices = [sc.pp.subsample(adata[adata.obs.perturbation_name == c], n_obs=100, copy=True).obs.index for c in adata.obs.perturbation_name.unique()]
adata
# batch_size=adata.shape[0]

AnnData object with n_obs × n_vars = 10600 × 19018
    obs: 'guide_identity', 'read_count', 'UMI_count', 'coverage', 'gemgroup', 'good_coverage', 'number_of_cells', 'guide_AHR', 'guide_ARID1A', 'guide_ARRDC3', 'guide_ATL1', 'guide_BAK1', 'guide_BCL2L11', 'guide_BCORL1', 'guide_BPGM', 'guide_C19orf26', 'guide_C3orf72', 'guide_CBFA2T3', 'guide_CBL', 'guide_CDKN1A', 'guide_CDKN1B', 'guide_CDKN1C', 'guide_CEBPA', 'guide_CEBPB', 'guide_CEBPE', 'guide_CELF2', 'guide_CITED1', 'guide_CKS1B', 'guide_CLDN6', 'guide_CNN1', 'guide_CNNM4', 'guide_COL1A1', 'guide_COL2A1', 'guide_CSRNP1', 'guide_DLX2', 'guide_DUSP9', 'guide_EGR1', 'guide_ELMSAN1', 'guide_ETS2', 'guide_FEV', 'guide_FOSB', 'guide_FOXA1', 'guide_FOXA3', 'guide_FOXF1', 'guide_FOXL2', 'guide_FOXO4', 'guide_GLB1L2', 'guide_HES7', 'guide_HK2', 'guide_HNF4A', 'guide_HOXA13', 'guide_HOXB9', 'guide_HOXC13', 'guide_IER5L', 'guide_IGDCC3', 'guide_IKZF3', 'guide_IRF1', 'guide_ISL2', 'guide_JUN', 'guide_KIAA1804', 'guide_KIF18B', 'guide_KIF2C', 'g

In [5]:
batch_size=2

create a dataloader and load your first batch (in this case all the cells):

In [6]:
scdl = model._make_data_loader(adata=adata, indices=list(range(adata.shape[0])), batch_size=batch_size)
batch = next(scdl.__iter__())

### Measure feature attribution with respect to posterior

Wrap `model.module.forward` because captum has an internal check that the inputs pass are tensors. scGen does not use a nb loss by default so we only have a single reconstruction output, `px`.

In [7]:
# inference output is in [0], generative output is in [1]
forw = lambda x, y, z: model.module.forward({'X': x, 'batch':y, 'labels':z}, compute_loss=False)[1]['px']

In [8]:
# example output
model.module.forward(batch, compute_loss=False)[1]['px'].shape

torch.Size([2, 19018])

create the ablator, containing the forward function inside of it:

In [9]:
ablator = FeatureAblation(forw)

Run the feature attribution function. The attribution below outputs two tensors. One ablates the gene features one by one, for every cell, and has shape (n_cells * n_ct_classes) * n_genes. The other ablates the (biological) batch variable, which it takes as a single, continuous variable (even though it is a one-hot encoded (n_datasets)-dimensional variable), and therefore ablates it once per cell, and outputs a (n_cells * n_ct_classes) * 1 shape tensor. We'll ignore that one.

In [ ]:
%%time
# per feature per output
attribution_map = ablator.attribute(
    (batch['X'], batch['batch'], batch['labels']),
    perturbations_per_eval=1,
    show_progress=True
)

Feature Ablation attribution:   1%|▉                                                                                                                                                                                  | 106/19021 [01:05<3:15:58,  1.61it/s]

In [9]:
attribution_map_genes = attribution_map[0] # take only the first tensor (explained above)

In [197]:
%%time
# # per feature per output
# attribution_map = ablator.attribute(
#     (batch['X'], batch['batch'], batch['labels']),
# #     target=['Macrophages']*batch['X'].shape[1],
#     feature_mask=(torch.tensor(feature_mask), torch.tensor(feature_mask), torch.tensor(feature_mask)))

# per feature (aggregation) (takes longer)
attribution_map = ablator.attribute(
    (batch['X'], batch['batch'], batch['labels']),
    perturbations_per_eval=1,
#     show_progress=True
)

CPU times: user 1h 30min 49s, sys: 33min 49s, total: 2h 4min 38s
Wall time: 7min 50s


In [175]:
[i.shape for i in attribution_map]

[torch.Size([10000, 2000]), torch.Size([10000, 1]), torch.Size([10000, 1])]

In [177]:
[i.shape for i in attribution_map]

[torch.Size([10000, 2000]), torch.Size([10000, 1]), torch.Size([10000, 1])]

Captum returns an attribution map of either `tensor(n_features * output_size, n_features), n_inputs` or ?

In [166]:
attribution_map[0].shape

torch.Size([10000, 2000])

In [115]:
attribution_map

(tensor([[-1.7431e-08, -1.7431e-08, -1.7431e-08,  ..., -1.7431e-08,
          -1.7431e-08, -1.7431e-08],
         [-1.0250e-10, -1.0250e-10, -1.0250e-10,  ..., -1.0250e-10,
          -1.0250e-10, -1.0250e-10],
         [-1.9924e-08, -1.9924e-08, -1.9924e-08,  ..., -1.9924e-08,
          -1.9924e-08, -1.9924e-08],
         ...,
         [-8.1410e-07, -8.1410e-07, -8.1410e-07,  ..., -8.1410e-07,
          -8.1410e-07, -8.1410e-07],
         [-9.7044e-01, -9.7044e-01, -9.7044e-01,  ..., -9.7044e-01,
          -9.7044e-01, -9.7044e-01],
         [-1.0539e-06, -1.0539e-06, -1.0539e-06,  ..., -1.0539e-06,
          -1.0539e-06, -1.0539e-06]]),
 tensor([[ 1.2044e-12],
         [-4.3241e-15],
         [ 1.4188e-12],
         [ 1.7650e-12],
         [ 5.3920e-14],
         [ 1.0132e-12],
         [ 1.1324e-12],
         [ 2.9206e-12],
         [ 1.7511e-16],
         [ 1.1511e-12],
         [ 2.4874e-13],
         [ 1.7375e-12],
         [ 8.7122e-13],
         [ 0.0000e+00],
         [-2.1124e

In [116]:
attribution_map[0].shape

torch.Size([140, 2000])

In [117]:
attribution_map[1].shape

torch.Size([140, 1])

In [16]:
ablator = FeatureAblation(model.module.forward)

reshape, such that n_cells\*n_classes is split into two dimensions ((n_cells x 28) becomes n_cells x 28)

In [10]:
attribution_map_genes_3d = attribution_map_genes.reshape((batch_size,28,2000))

For each class, calculate mean only across cells of that class (= cell type), ignore other cells. Then take only the feature importances for that particular class. Reasoning: we want to learn which features were important for classifing a cell of cell type a *as* cell type a, and not the features that made the model *not* classify it as cell type a (the latter would give negative markers rather than positive ones). 

In [11]:
means = pd.DataFrame(index=adata.var_names,columns=ct_names)

In [12]:
for ct in batch['labels'].unique():
    ct_float = ct.item()
    ct_int = int(ct_float)
    ct_indexing = (batch['labels'] == ct_float).reshape(-1)
    means.iloc[:,ct_int] = torch.mean(attribution_map_genes_3d[ct_indexing,ct_int,:],dim=0)

### measure against latent (to do)

## Store results:

In [13]:
means.to_csv("../outputs/ablation/task2.csv")